In [20]:
import numpy as np # Pour la manipulation de tableaux
import tensorflow as tf
import pandas as pd # Pour manipuler des DataFrames pandas

import matplotlib.pyplot as plt # Pour l'affichage d'images
from matplotlib import cm # Pour importer de nouvelles cartes de couleur
%matplotlib inline


from tensorflow.keras.layers import Input, Dense #Pour instancier une couche Dense et une d'Input
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [92]:
df = pd.read_csv('Sources/df_final_rolling_ready_50_with_bookodds.csv')

In [93]:
# Encodage
le = LabelEncoder()
le.fit(pd.concat([df['player_1_name'], df['player_2_name']], axis=0))
df['player_1_name_encoded'] = le.transform(df['player_1_name'])
df['player_2_name_encoded'] = le.transform(df['player_2_name'])
# Suppression player_1_name & player_2
df.drop(['player_1_name', 'player_2_name'], axis=1, inplace=True)
# Dichotomisation
df = pd.get_dummies(df)
# Split
nb_rows_train = int(round(len(df)*0.7,0))
X_train = df.drop('player_1_win', axis=1)[:nb_rows_train]
y_train = df['player_1_win'][:nb_rows_train]
X_test = df.drop('player_1_win', axis=1)[nb_rows_train:]
y_test = df['player_1_win'][nb_rows_train:]
# Standardisation
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [94]:
model = Sequential()
model.add(Input(shape = (123)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [95]:
#lr_plateau = ReduceLROnPlateau(monitor='val_loss',
#                                        patience=2,
#                                         verbose=2,
#                                         mode='min')
es = EarlyStopping(monitor='val_accuracy',
                                         patience=3,
                                         mode='max',
                                         restore_best_weights=True)

In [96]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = "adam",
              metrics = ["accuracy"])

In [97]:
model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test), callbacks=[es])

Epoch 1/100


C:\Users\enovap\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


628/628 [==============================] - 1s 2ms/step - loss: 0.6086 - accuracy: 0.6634 - val_loss: 0.6050 - val_accuracy: 0.6629
Epoch 2/100
628/628 [==============================] - 1s 1ms/step - loss: 0.5750 - accuracy: 0.6929 - val_loss: 0.6047 - val_accuracy: 0.6658
Epoch 3/100
628/628 [==============================] - 1s 1ms/step - loss: 0.5653 - accuracy: 0.7028 - val_loss: 0.6075 - val_accuracy: 0.6663
Epoch 4/100
628/628 [==============================] - 1s 2ms/step - loss: 0.5606 - accuracy: 0.7040 - val_loss: 0.6055 - val_accuracy: 0.6716
Epoch 5/100
628/628 [==============================] - 1s 2ms/step - loss: 0.5544 - accuracy: 0.7103 - val_loss: 0.6081 - val_accuracy: 0.6638
Epoch 6/100
628/628 [==============================] - 1s 1ms/step - loss: 0.5490 - accuracy: 0.7121 - val_loss: 0.6189 - val_accuracy: 0.6665
Epoch 7/100
628/628 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7167 - val_loss: 0.6181 - val_accuracy: 0.6521


In [98]:
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Accuracy", accuracy)

269/269 [==============================] - 0s 845us/step - loss: 0.6055 - accuracy: 0.6716
Accuracy 0.6716278791427612


In [99]:
y_pred = model.predict(X_test_scaled)

269/269 [==============================] - 0s 707us/step


In [100]:
y_pred = np.where(y_pred>0.5,1,0)

In [101]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.67      0.68      0.68      4351
           1       0.67      0.66      0.66      4249

    accuracy                           0.67      8600
   macro avg       0.67      0.67      0.67      8600
weighted avg       0.67      0.67      0.67      8600

[[2977 1374]
 [1450 2799]]
